In [0]:
file_type = 'parquet'

songs = spark.read.parquet("s3://full-stack-bigdata-datasets/Big_Data/YOUTUBE/items_selected.parquet")
songs.printSchema()
songs.count(), len(songs.columns)

root
 |-- contentDetails_duration: string (nullable = true)
 |-- id: string (nullable = true)
 |-- snippet_channelId: string (nullable = true)
 |-- snippet_channelTitle: string (nullable = true)
 |-- snippet_publishedAt: string (nullable = true)
 |-- snippet_title: string (nullable = true)
 |-- statistics_commentCount: double (nullable = true)
 |-- statistics_dislikeCount: double (nullable = true)
 |-- statistics_viewCount: long (nullable = true)

Out[1]: (3907, 9)

In [0]:
songs.describe().toPandas()

,summary,contentDetails_duration,id,snippet_channelId,snippet_channelTitle,snippet_publishedAt,snippet_title,statistics_commentCount,statistics_dislikeCount,statistics_viewCount
0,count,3907,3907,3907,3907,3907,3907,3880,3893,3907
1,mean,None,None,None,7.296272962E9,None,None,6134.878865979382,4669.332391471872,2.0763860959815715E7
2,stddev,None,None,None,0.0,None,None,38535.66393506117,34768.35562984628,1.2295665130505532E8
3,min,PT10M,--RRkree8kM,UC--yIemFNSgwQ0JxyYsABAQ,!!! (Chk Chk Chk),2005-11-02T22:02:48Z,!!! - One Girl/One Boy,0.0,0.0,78
4,max,PT9M7S,zzrNI66FUWE,UCzz_ERveHOdZve_Qsthwhnw,さむい者おげし,2019-11-04T14:55:48Z,📺 Danakil - Le rêve [Official Video],1057699.0,1373532.0,3058853981


In [0]:
import pandas as pd
from pyspark.sql import functions as F

songs.select(*(F.sum(F.col(c).isNull().cast('int')).alias(c) for c in songs.columns)) \
     .toPandas().set_index(pd.Index(['missing']))

,contentDetails_duration,id,snippet_channelId,snippet_channelTitle,snippet_publishedAt,snippet_title,statistics_commentCount,statistics_dislikeCount,statistics_viewCount
missing,0,0,0,0,0,0,27,14,0


In [0]:
songs.select('snippet_channelTitle', 'snippet_title', 'statistics_viewCount') \
     .orderBy(F.desc('statistics_viewCount')).limit(5).toPandas()

,snippet_channelTitle,snippet_title,statistics_viewCount
0,OneRepublicVEVO,OneRepublic - Counting Stars (Official Music V...,3058853981
1,Passenger,Passenger | Let Her Go (Official Video),2789082191
2,EminemVEVO,Eminem - Love The Way You Lie ft. Rihanna,2064352955
3,AviciiOfficialVEVO,Avicii - Wake Me Up (Official Video),1963215194
4,gotyemusic,Gotye - Somebody That I Used To Know (feat. Ki...,1506602164


In [0]:
exprs = (
  F.sum('statistics_viewCount').alias('total_viewCount'),
  F.mean('statistics_viewCount').alias('mean_viewCount'),
  F.min('statistics_viewCount').alias('min_viewCount'),
  F.max('statistics_viewCount').alias('max_viewCount'),
  F.stddev('statistics_viewCount').alias('stdViewCount'),
  F.count('*').alias('songsCount'),
)

channels = songs.groupBy('snippet_channelTitle').agg(*exprs)
channels.limit(5).toPandas()

,snippet_channelTitle,total_viewCount,mean_viewCount,min_viewCount,max_viewCount,stdViewCount,songsCount
0,"Being part of something special, makes you spe...",67464186,16866046.5,13069025,27567026,7.139327e+06,4
1,Grimes,66954706,33477353.0,1966671,64988035,4.456283e+07,2
2,Sargent House,123884,123884.0,123884,123884,NaN,1
3,WildBeastsVEVO,581479,581479.0,581479,581479,NaN,1
4,ParetoPark,3083,3083.0,3083,3083,NaN,1


In [0]:
channels.orderBy(F.desc('mean_viewCount')).limit(5).toPandas()

,snippet_channelTitle,total_viewCount,mean_viewCount,min_viewCount,max_viewCount,stdViewCount,songsCount
0,OneRepublicVEVO,3058853981,3.058854e+09,3058853981,3058853981,NaN,1
1,Passenger,2789082191,2.789082e+09,2789082191,2789082191,NaN,1
2,gotyemusic,1506602164,1.506602e+09,1506602164,1506602164,NaN,1
3,GunsNRosesVEVO,1436168604,1.436169e+09,1436168604,1436168604,NaN,1
4,PitbullVEVO,1231470918,1.231471e+09,1231470918,1231470918,NaN,1


In [0]:
channels.orderBy(F.desc('max_viewCount')).limit(5).toPandas()

,snippet_channelTitle,total_viewCount,mean_viewCount,min_viewCount,max_viewCount,stdViewCount,songsCount
0,OneRepublicVEVO,3058853981,3.058854e+09,3058853981,3058853981,NaN,1
1,Passenger,2789082191,2.789082e+09,2789082191,2789082191,NaN,1
2,EminemVEVO,4181548740,8.363097e+08,35244992,2064352955,8.032428e+08,5
3,AviciiOfficialVEVO,2881230160,7.203075e+08,73376915,1963215194,8.477492e+08,4
4,gotyemusic,1506602164,1.506602e+09,1506602164,1506602164,NaN,1


In [0]:
channels.orderBy(F.desc('total_viewCount')).limit(5).toPandas()

,snippet_channelTitle,total_viewCount,mean_viewCount,min_viewCount,max_viewCount,stdViewCount,songsCount
0,EminemVEVO,4181548740,8.363097e+08,35244992,2064352955,8.032428e+08,5
1,OneRepublicVEVO,3058853981,3.058854e+09,3058853981,3058853981,NaN,1
2,AviciiOfficialVEVO,2881230160,7.203075e+08,73376915,1963215194,8.477492e+08,4
3,Passenger,2789082191,2.789082e+09,2789082191,2789082191,NaN,1
4,Macklemore LLC,2541772489,8.472575e+08,252792552,1477980957,6.133985e+08,3


In [0]:
channels.orderBy(F.desc('songsCount')).limit(5).toPandas()

,snippet_channelTitle,total_viewCount,mean_viewCount,min_viewCount,max_viewCount,stdViewCount,songsCount
0,TheSoundYouNeed,1713339950,4.283350e+07,481371,512730266,1.090023e+08,40
1,Majestic Casual,271966905,7.770483e+06,329645,54628130,1.400784e+07,35
2,Spinnin' Records,2256309801,1.253505e+08,182933,1425286294,3.455796e+08,18
3,MrSuicideSheep,149500698,8.305594e+06,539727,96501569,2.239703e+07,18
4,FAUVE,66064873,3.886169e+06,223988,12990703,4.195480e+06,17


In [0]:
display(channels.select(F.log('mean_viewCount'), F.log('songsCount')))



ln(mean_viewCount),ln(songsCount)
16.640813076138894,1.3862943611198906
17.32637973833333,0.6931471805599453
11.727100922878089,0.0
13.273330136789179,0.0
8.03365842788615,0.0
16.002462889304873,0.0
10.482730882413174,1.3862943611198906
7.428333194190806,0.0
7.247080584585756,0.0
18.212742405255103,0.0
